# Extract the Personal Identifiable Information (PII) using Watson NLP

<h2>Use Case</h2>

This notebook demonstrates how to extract PII entities using Watson NLP Pre-trained  models also demonstrates how to prepare custom train models. PII extraction is the process of identifying and extracting personal information from a document or dataset. This information can include names, addresses, phone numbers, email addresses, Social Security numbers, Credit Card number, and other types of information that can be used to identify an individual. 


<h2>What you'll learn in this notebook</h2>

Watson NLP offers Pre-trained Models for various NLP tasks also provides fine-tune functionality for custom training. This notebooks shows:

* <b>RBR</b>:  A Rule-Based Reasoner (RBR) in NLP works by using a set of predefined rules to process and understand natural language input. These rules are used to identify specific patterns or structures in the input text and determine the meaning of the text based on those patterns.


* <b>BILSTM</b>: the BiLSTM network would take the preprocessed text as input and learn to identify patterns and relationships between words that are indicative of PII data. The BiLSTM network would then output a probability score for each word in the text, indicating the likelihood that the word is part of a PII entity. The BiLSTM network may also be trained to recognize specific entities such as names, addresses, phone numbers, email addresses, etc.



## Table of Contents


1. [Before you start](#beforeYouStart)
1. [Load Entity PII Models](#LoadModel)
1. [Load PII XLSX Dataset from Data Assets](#Loaddata)
1. [Watson NLP Models](#NLPModels)    
   1. [BiLSTM Pretrained](#BILSTMPre)
   1. [RBR Pretrained](#RBRPre)
1. [Testing Usecase](#Testing)  
   1.  [RBR Model Testing for Caredit card Extraction, URL and Email](#5.1)
       1. [RBR Stock - URL Extraction](#5.1.1)
       1. [Combine RBR PII and RBR Stock (URL)](#5.1.2)
   1. [EmailAddress Extraction using RBR](#5.3)
1. [Summary](#summary)

<a id="beforeYouStart"></a>
### 1. Before you start


<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.x environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

In [3]:
import json
import pandas as pd
import watson_nlp
from watson_nlp import data_model as dm
from watson_nlp.toolkit.entity_mentions_utils import prepare_train_from_json

In [4]:
# Silence Tensorflow warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

<a id="LoadModel"></a>
### 2. Load Entity PII Models

The function "watson_nlp.load()" is used to download a pre-trained NLP model, such as IBM Watson NLP, which can then be used to process and analyze text data. This function allows you to quickly and easily integrate advanced NLP capabilities into your application, without having to train a model from scratch. By downloading a pre-trained model, you can save time and resources, and benefit from the expertise of the model developers.

In [5]:
# Load a syntax model to split the text into sentences and tokens
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))
# Load bilstm model in WatsonNLP
bilstm_model = watson_nlp.load(watson_nlp.download('entity-mentions_bilstm_en_pii'))
# Load rbr model in WatsonNLP
rbr_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_multi_pii'))

<a id="Loaddata"></a>
### 3. Load PII XLSX Dataset from Data Assets

Load the sample data from s3. This data contains Name, SSN and credit card numbers.

In [6]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='o0avUc3SDky2d6pNzjuewCSTPPX7tQNz6BKKvL37nBL3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'watsoncore-donotdelete-pr-olkxvfa8bk0pb1'
object_key = '10-MB-Test.xlsx'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']

df = pd.read_excel(body.read())
df = df.dropna()
df.head()

,First and Last Name,SSN,Credit Card Number,First and Last Name.1,SSN.1,Credit Card Number.1,First and Last Name.2,SSN.2,Credit Card Number.2,First and Last Name.3,...,Credit Card Number.3,First and Last Name.4,SSN.4,Credit Card Number.4,First and Last Name.5,SSN.5,Credit Card Number.5,First and Last Name.6,SSN.6,Credit Card Number.6
1,Robert Aragon,489-36-8350,4929-3813-3266-4295,Robert Aragon,489-36-8351,4929-3813-3266-4296,Robert Aragon,489-36-8352,4929-3813-3266-4297,Robert Aragon,...,4929-3813-3266-4298,Robert Aragon,489-36-8354,4929-3813-3266-4299,Robert Aragon,489-36-8355,4929-3813-3266-4300,Robert Aragon,489-36-8355,4929-3813-3266-4300
2,Ashley Borden,514-14-8905,5370-4638-8881-3020,Ashley Borden,514-14-8906,5370-4638-8881-3021,Ashley Borden,514-14-8907,5370-4638-8881-3022,Ashley Borden,...,5370-4638-8881-3023,Ashley Borden,514-14-8909,5370-4638-8881-3024,Ashley Borden,514-14-8910,5370-4638-8881-3025,Ashley Borden,514-14-8910,5370-4638-8881-3025
3,Thomas Conley,690-05-5315,4916-4811-5814-8111,Thomas Conley,690-05-5316,4916-4811-5814-8112,Thomas Conley,690-05-5317,4916-4811-5814-8113,Thomas Conley,...,4916-4811-5814-8114,Thomas Conley,690-05-5319,4916-4811-5814-8115,Thomas Conley,690-05-5320,4916-4811-5814-8116,Thomas Conley,690-05-5320,4916-4811-5814-8116
4,Susan Davis,421-37-1396,4916-4034-9269-8783,Susan Davis,421-37-1397,4916-4034-9269-8784,Susan Davis,421-37-1398,4916-4034-9269-8785,Susan Davis,...,4916-4034-9269-8786,Susan Davis,421-37-1400,4916-4034-9269-8787,Susan Davis,421-37-1401,4916-4034-9269-8788,Susan Davis,421-37-1401,4916-4034-9269-8788
5,Christopher Diaz,458-02-6124,5299-1561-5689-1938,Christopher Diaz,458-02-6125,5299-1561-5689-1939,Christopher Diaz,458-02-6126,5299-1561-5689-1940,Christopher Diaz,...,5299-1561-5689-1941,Christopher Diaz,458-02-6128,5299-1561-5689-1942,Christopher Diaz,458-02-6129,5299-1561-5689-1943,Christopher Diaz,458-02-6129,5299-1561-5689-1943


<a id="NLPModels"></a>
### 4.  Watson NLP Models

<a id="BILSTMPre"></a>
### 4. 1 BiLSTM Pretrained

The term "pretrained" refers to a pre-trained BiLSTM model, which has already been trained on a large corpus of text data and can be fine-tuned or used as is for specific NLP tasks, such as sentiment analysis, named entity recognition, and so on. By using a pretrained BiLSTM model, you can leverage the knowledge learned from the training data to quickly build NLP applications with improved accuracy.

In [7]:
#help(bilstm_model)

In [10]:
text = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][1], df['Credit Card Number'][1])

In [15]:
#Test Pretrained bilstm_model model in WatsonNLP
syntax_result = syntax_model.run(text)
bilstm_result = bilstm_model.run(syntax_result)

for i in bilstm_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  Robert Aragon   Type:  Person


<a id="RBRPre"></a>

### 4.2 RBR Pretrained


A pretrained rule-based model is a model that has already been trained on a large corpus of text data and has a set of predefined rules for processing text data. By using a pretrained rule-based model, you can leverage the knowledge learned from the training data to quickly build NLP applications with improved accuracy.

In [ ]:
text1 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][1], "` 378282246310005 `")
text2 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][2], "`378282246310005 `")
text3 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][3], "` 378282246310005 `")
text4 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][4], "` 5555555555554444 `")
text5 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][5], "`5555555555554444 `")
text6 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][6], "` 5555-5555-5555-4444 `")
text7 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][7], "`5555-5555-5555-4444 `")
text8 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][8], "` 5555-5555-5555-4444`")

In [ ]:
all_test=[text1,text2,text3,text4,text5,text6,text7,text8]

In [23]:
# help(rbr_model)

In [26]:
# Test the pretrain
for test in all_test:
    rbr_result = rbr_model.run(test, language_code='en')

    for i in rbr_result.mentions:
        print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  514-14-8905     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  690-05-5315     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  421-37-1396     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  458-02-6124     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  612-20-6832     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  300-62-3266     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  66

<a id="Testing"></a>

## 5. Testing Usecase

<a id="5.1"></a>
### 5.1 RBR Model Testing for Caredit card Extraction, URL and Email

<a id="5.1.1"></a>
#### 5.1.1 RBR Stock (URL Extraction)

In [30]:
#test dataset for URL extraction 
test1 = "My name is Robert\xa0Aragon, and my social security number is 489-36-8350. Here's the number to my Visa credit card, http://www.example.com/page_id=5555555555554444"

In [31]:
#Test Pretrained rbr stock model in WatsonNLP
rbr_ent_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_en_stock'))
rbr_ent_result = rbr_ent_model.run(test1)

for i in rbr_ent_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  http://www.example.com/page_id=5555555555554444 Type:  URL


In [32]:
#Test Pretrained rbr PII model in WatsonNLP

rbr_result_pii = rbr_model.run(test1, language_code='en')
rbr_result_pii

for i in rbr_result_pii.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US


<a id="5.1.2"></a>
#### 5.1.2 Combine RBR PII and RBR Stock (URL)

In [33]:
# combine to be one object
combined_mentions = rbr_ent_result + rbr_result_pii
combined_mentions

for i in combined_mentions.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US
Text:  http://www.example.com/page_id=5555555555554444 Type:  URL


<a id="5.2"></a>
### 5.2 EmailAddress Extraction using RBR 

In [47]:
#test dataset for EMail extraction 
text = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s. My Email id is %s" % (df['First and Last Name'][1], df['SSN'][1], df['Credit Card Number'][1],"sample.Email@gmail.com")
text

"My name is Robert\xa0Aragon, and my social security number is 489-36-8350. Here's the number to my Visa credit card, 4929-3813-3266-4295. My Email id is sample.Email@gmail.com"

In [48]:
#Test Pretrained rbr PII model in WatsonNLP

rbr_result_pii = rbr_model.run(text, language_code='en')
rbr_result_pii

for i in rbr_result_pii.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US
Text:  4929-3813-3266-4295 Type:  BankAccountNumber.CreditCardNumber.Visa
Text:  sample.Email@gmail.com Type:  EmailAddress


In [49]:
#Test Pretrained rbr stock model in WatsonNLP
rbr_ent_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_en_stock'))
rbr_ent_result = rbr_ent_model.run(text)

for i in rbr_ent_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  4929-3813-3266  Type:  PhoneNumber
Text:  sample.Email@gmail.com Type:  EmailAddress


<a id="summary"></a>
## 6. Summary

<span style="color:blue">This notebook shows you how to use the Watson NLP library to:
1. Extract PII using Pre-trained Models
</span>

Please note that this content is made available by IBM Build Lab to foster Embedded AI technology adoption. The content may include systems & methods pending patent with USPTO and protected under US Patent Laws. For redistribution of this content, IBM will use release process. For any questions please log an issue in the GitHub.

Developed by IBM Build Lab

Copyright - 2023 IBM Corporation